In [1]:
import papermill
from utils import *
from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AdamW
from torch.optim import Adam
import time

In [2]:
pairs = [(48,80),(96,40),(24,160),(12,320)]

In [3]:
iterations = len(pairs) # number of parameter combinations to try

In [4]:
# HYPERPARAMETER CANDIDATES - try out whatever tickles your fancy

possible_params = {
    # general training parameters
    'LEARNING_RATE':[1e-4, 3e-4, 5e-4, 6e-4, 7e-4],
    #'SCHEDULER_CLASS':[get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_constant_schedule_with_warmup],
    'TEMPERATURE':[9],
    #'OPTIMIZER':['AdamW', 'AdamW(weight_decay=0.01)', 'Adam(weight_decay=0.01)'],
    'BATCH_SIZE':[256],
    # switch transformer - student config
    'D_MODEL':[768],
    'HEADS':[12],
    'D_FF':[768, 384, 192, 96],
    'N_LAYERS':[3],
    'N_EXPERTS':[16,24,32,40,48,56,64,72,80,88]  
}

params = generate_random_params(possible_params)

In [5]:
params['LEARNING_RATE'] = 5e-4
params['N_LAYERS'] = 2
params['NUM_STEPS'] = 10000
params['TRAINING_DATA'] = 10000000
params['BATCH_SIZE'] = 128

In [6]:
start = time.time()
for i in range(iterations):
    params['D_FF'] = pairs[i][0]
    params['N_EXPERTS'] = pairs[i][1]
    print('Iteration {}\'s params: {}\n'.format(i+1,params))
    papermill.execute_notebook(
    'distillation.ipynb',
    'outputs/distillation_{}.ipynb'.format(i+1),
    kernel_name='python3',
    parameters=params
    )
    end = time.time()
    print('Time (s) passed since last iteration: ',end-start)
    start = end
    

Iteration 1's params: {'LEARNING_RATE': 0.0005, 'TEMPERATURE': 9, 'BATCH_SIZE': 128, 'D_MODEL': 768, 'HEADS': 12, 'D_FF': 48, 'N_LAYERS': 2, 'N_EXPERTS': 80, 'NUM_STEPS': 10000, 'TRAINING_DATA': 10000000}



Executing:   0%|          | 0/47 [00:00<?, ?cell/s]

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [26]":
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<ipython-input-26-cde54dc8f34f> in <module>
     20 with distiller:
     21     distiller.train(optimizer,dataloader, num_steps=NUM_STEPS, 
---> 22     scheduler_class=SCHEDULER_CLASS, scheduler_args = scheduler_args, callback=callback_fun)
     23 end = time.time()
     24 

~/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/textbrewer-0.2.1.post1-py3.6.egg/textbrewer/distiller_basic.py in train(self, optimizer, dataloader, num_epochs, scheduler_class, scheduler_args, scheduler, max_grad_norm, num_steps, callback, batch_postprocessor, **args)
    298         assert not (num_epochs is None and num_steps is None)
    299         if num_steps is not None:
--> 300             self.train_with_num_st

In [ ]:
!jupyter kernelspec list

In [7]:
%cd /home/ec2-user/anaconda3/envs/pytorch_p36

/home/ec2-user/anaconda3/envs/pytorch_p36


In [7]:
%reset -f